### Import

In [38]:
import os
import pandas as pd
import numpy as np
from pandas import Categorical, get_dummies
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

### Load data

In [2]:
print(os.listdir('./datasets/housing'))

housing = pd.read_csv('./datasets/housing/housing.csv')
housing.head()

['housing.csv', 'housing.tgz', 'README.md']


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Split data

In [59]:
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
#print(housing.income_cat)
# note this code. It's so nice that series also have where!!!
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
# housing.income_cat.hist(bins = np.arange(6)-0.5);
# plt.hist(housing.income_cat)

from sklearn.model_selection import StratifiedShuffleSplit

mysplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# since n_splits=1 we have only one list for train_index, one list for test_index
for train_index, test_index in mysplit.split(housing, housing['income_cat']):
    train_data = housing.loc[train_index]
    test_data = housing.loc[test_index]

xtrain = train_data.drop(columns=['median_house_value', 'income_cat'])
ytrain = train_data['median_house_value']

xtest = test_data.drop(columns=['median_house_value', "income_cat"])
ytest = test_data['median_house_value']
print(xtrain.shape)

(16512, 9)


### Handling numeric features

For imputing the missing values for numeric features, use only knowledge from train set. This mean "fit" using the train set and then "transform" for both train and test set

In [4]:
# Selector to just select the data
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# column numbers of some attributes
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6    
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self # nothing else to do
    
    def transform(self, X, y=None):
        # 2 features are always added
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        # one feature can be added or not based on the boolean parameter
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            # np.c_ Translates slice objects to concatenation along the second axis.
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        
        else:
            #print(X.shape)
            #print(rooms_per_household.shape)
            #print(np.c_[X, rooms_per_household, population_per_household].shape)
            return np.c_[X, rooms_per_household, population_per_household]

In [5]:
num_att = list(xtrain.drop(columns=["ocean_proximity"]))

num_pipeline = Pipeline([("selector", DataFrameSelector(num_att)), 
                         ("imputer", Imputer(strategy="median")),
                        ("add_attributes", CombinedAttributesAdder()),
                        ("std_scaler", StandardScaler())])

num_pipeline.fit(xtrain)
xtrain_num = num_pipeline.transform(xtrain)
xtest_num = num_pipeline.transform(xtest)
print(xtrain_num.shape)
print(xtest_num.shape)

(16512, 11)
(4128, 11)


### Handling categorical features

In order to avoid the situation that some categories exist in the test set, but not in the train set, I would prefer not to use pipeline here

In [60]:
# get all possible categories
cat_cols = ["ocean_proximity"]
for col in cat_cols:
    possible_categories = housing[col].unique().tolist()
    xtrain[col] = pd.Categorical(xtrain[col], categories=possible_categories)
    xtest[col] = pd.Categorical(xtest[col], categories=possible_categories)

xtrain_cat = pd.get_dummies(xtrain[cat_cols])
xtest_cat = pd.get_dummies(xtest[cat_cols])
print(xtrain_cat.shape)
print(xtest_cat.shape)
print(list(xtrain_cat))

(16512, 5)
(4128, 5)
['ocean_proximity_NEAR BAY', 'ocean_proximity_<1H OCEAN', 'ocean_proximity_INLAND', 'ocean_proximity_NEAR OCEAN', 'ocean_proximity_ISLAND']


### Joining numeric and categorical

In [14]:
xtrain_trans = np.concatenate((xtrain_num, xtrain_cat), axis=1)
xtest_trans = np.concatenate((xtest_num, xtest_cat), axis=1)
print(xtrain_trans.shape)
print(xtest_trans.shape)

(16512, 16)
(4128, 16)


### Select model using cross validation

In [35]:
from sklearn.model_selection import cross_validate

# using mean squared error and r2_score as metrics for cross validation training
scoring = {'mse': 'neg_mean_squared_error',
           'r2': 'r2'
           }

def print_scores(scores):
    """
    print out scores
    :param scores:
    :return:
    """
    print("train rmse: %.2f" % np.sqrt(-scores["train_mse"]).mean())
    print("train r2 score: %.2f" % scores["train_r2"].mean())
    print("validation rmse: %.2f" % np.sqrt(-scores["test_mse"]).mean())
    print("validation score: %.2f" % scores["test_r2"].mean())
    

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

#### Linear model

In [37]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression()

lin_scores = cross_validate(lin, xtrain_trans, ytrain, scoring=scoring,
                        cv=10, return_train_score=True)
print("\n\nLinear regression:\n")

# save model
joblib.dump(lin, "linear_regressor.pkl")
joblib.dump(lin_scores, "linear_scores.pkl")

print_scores(lin_scores)



Linear regression:

train rmse: 68605.81
train r2 score: 0.65
validation rmse: 69052.46
validation score: 0.64


#### Decision Tree

In [31]:
from sklearn.tree import DecisionTreeRegressor

dec_tree = DecisionTreeRegressor()
dt_scores = cross_validate(dec_tree, xtrain_trans, ytrain, scoring=scoring,
                        cv=10, return_train_score=True)
# save model
joblib.dump(dec_tree, "dtree_regressor.pkl")
joblib.dump(dt_scores, "dtree_scores.pkl")

print("\n\nDecision Tree:\n")
print_scores(dt_scores)



Decision Tree:

train rmse: 0.00
train r2 score: 1.00
validation rmse: 71296.64
validation score: 0.62


#### RandomForest

Random Forests work by training many Decision Trees on random subsets of the features, then averaging out their predictions. Building a model on top of many other models is called Ensemble Learning, and it is often a great way to push ML algorithms even further.

In [32]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf_scores = cross_validate(rf, xtrain_trans, ytrain, scoring=scoring,
                        cv=10, return_train_score=True)

# save model
joblib.dump(rf, "rf_regressor.pkl")
joblib.dump(rf_scores, "rf_scores.pkl")

print("\n\nRandom Forest:\n")
print_scores(rf_scores)



Random Forest:

train rmse: 22463.36
train r2 score: 0.96
validation rmse: 52834.54
validation score: 0.79


#### XGBoost

In [39]:
from xgboost import XGBRegressor


xb = XGBRegressor()
xb_scores = cross_validate(xb, xtrain_trans, ytrain, scoring=scoring,
                        cv=10, return_train_score=True)

# save model
joblib.dump(xb, "xb_regressor.pkl")
joblib.dump(xb_scores, "xb_scores.pkl")

print("\n\nRandom Forest:\n")
print_scores(xb_scores)



Random Forest:

train rmse: 50731.24
train r2 score: 0.81
validation rmse: 53109.23
validation score: 0.79


### Fine-tune model

#### Using GridSearch

In [50]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_estimators":[3, 10, 30], "max_features":[2,4,6,8]},
    {"bootstrap":[False], "n_estimators":[3, 10], "max_features":[2,3,4]}
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)
grid_search.fit(xtrain_trans, ytrain)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [51]:
print(grid_search.best_params_)

{'max_features': 8, 'n_estimators': 30}


In [52]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [54]:
cvres = grid_search.cv_results_

print(cvres.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_features', 'param_n_estimators', 'param_bootstrap', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score'])


In [55]:
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

64425.23872443877 {'max_features': 2, 'n_estimators': 3}
55257.24295868718 {'max_features': 2, 'n_estimators': 10}
52797.815600453396 {'max_features': 2, 'n_estimators': 30}
60619.728379472195 {'max_features': 4, 'n_estimators': 3}
53154.7169911524 {'max_features': 4, 'n_estimators': 10}
50569.60484633807 {'max_features': 4, 'n_estimators': 30}
59330.117203289235 {'max_features': 6, 'n_estimators': 3}
52071.51052192515 {'max_features': 6, 'n_estimators': 10}
50141.734947929916 {'max_features': 6, 'n_estimators': 30}
59244.32038081152 {'max_features': 8, 'n_estimators': 3}
51818.80762530009 {'max_features': 8, 'n_estimators': 10}
50007.79571275452 {'max_features': 8, 'n_estimators': 30}
62187.00579667948 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54730.672533001416 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
58379.93836315784 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52547.82376020723 {'bootstrap': False, 'max_features': 3, 'n_estimators'

#### Using random_search

#### Analyze the best model and their errors

In [57]:
feature_importances = grid_search.best_estimator_.feature_importances_

In [58]:
print(feature_importances)

[6.74265071e-02 6.27745324e-02 4.45390393e-02 1.47554592e-02
 1.49211945e-02 1.47899183e-02 1.40352329e-02 3.90621992e-01
 3.58154821e-02 1.09605443e-01 6.51105552e-02 1.97445532e-03
 4.55290592e-03 1.55883816e-01 3.11623755e-03 7.72296397e-05]


Let’s display these importance scores next to their corresponding attribute names

In [61]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(xtrain_cat)
attributes = num_att + extra_attribs + cat_one_hot_attribs 

In [62]:
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3906219920329171, 'median_income'),
 (0.15588381564032533, 'ocean_proximity_INLAND'),
 (0.10960544271748147, 'pop_per_hhold'),
 (0.06742650713076309, 'longitude'),
 (0.06511055522255164, 'bedrooms_per_room'),
 (0.06277453240885793, 'latitude'),
 (0.044539039349090453, 'housing_median_age'),
 (0.03581548212058292, 'rooms_per_hhold'),
 (0.014921194485299282, 'total_bedrooms'),
 (0.014789918343657676, 'population'),
 (0.014755459196293902, 'total_rooms'),
 (0.014035232925918881, 'households'),
 (0.004552905920272784, 'ocean_proximity_<1H OCEAN'),
 (0.003116237549921739, 'ocean_proximity_NEAR OCEAN'),
 (0.0019744553163461155, 'ocean_proximity_NEAR BAY'),
 (7.722963971963629e-05, 'ocean_proximity_ISLAND')]

### Make prediction on the test set

In [66]:
final_model = grid_search.best_estimator_

ypred_test = final_model.predict(xtest_trans)

print(np.sqrt(mean_squared_error(ytest, ypred_test)))
print(r2_score(ytest, ypred_test))

47839.40793014073
0.8243806205835597
